In [85]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported')

Libraries imported


In [110]:
# scrape the Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe
other_path = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(other_path)[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [111]:
df = df[df['Borough'] != 'Not assigned']

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [112]:
df['Neighborhood'] = df['Neighborhood'].fillna(df['Borough'])

In [113]:
df.reset_index(inplace = True, drop = True) 
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [95]:
df.shape

(103, 3)

In [107]:
geoSpatialData_path = "http://cocl.us/Geospatial_data"
geoSpatialData = pd.read_csv(geoSpatialData_path, header = None, index_col = None)
geoSpatialData.columns = geoSpatialData.iloc[0]
geoSpatialData = geoSpatialData.drop(geoSpatialData.index[0])
geoSpatialData.head()

,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174
5,M1H,43.773136,-79.2394761


In [114]:
df = pd.merge(df, geoSpatialData, on = 'Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.4647633
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938
